# 00 测试AkShare API

验证AkShare的futures_zh_daily_sina API是否正常工作

In [ ]:
import akshare as ak
import pandas as pd
import numpy as np
from datetime import datetime
import time

print(f"AkShare版本: {ak.__version__}")
print(f"Pandas版本: {pd.__version__}")

## 1. 测试单个品种获取

In [ ]:
# 测试获取RB0 (螺纹钢)
symbol = 'RB0'
print(f"获取 {symbol} 数据...")

try:
    df = ak.futures_zh_daily_sina(symbol=symbol)
    print(f"✓ 成功获取 {len(df)} 条数据")
    print(f"\n数据列: {list(df.columns)}")
    print(f"\n日期范围: {df['date'].min()} 至 {df['date'].max()}")
    
    # 显示数据样本
    print("\n最新5条数据:")
    display(df.tail())
    
except Exception as e:
    print(f"✗ 错误: {e}")

## 2. 测试多个品种

In [ ]:
# 测试黑色系品种
test_symbols = {
    'RB0': '螺纹钢',
    'HC0': '热卷',
    'I0': '铁矿石',
    'SF0': '硅铁',
    'SM0': '锰硅',
    'SS0': '不锈钢'
}

results = []

for symbol, name in test_symbols.items():
    try:
        print(f"获取 {symbol} ({name})...", end=' ')
        df = ak.futures_zh_daily_sina(symbol=symbol)
        
        if df is not None and not df.empty:
            results.append({
                '品种代码': symbol,
                '品种名称': name,
                '数据条数': len(df),
                '开始日期': df['date'].min(),
                '结束日期': df['date'].max(),
                '状态': '成功'
            })
            print(f"✓ {len(df)} 条")
        else:
            results.append({
                '品种代码': symbol,
                '品种名称': name,
                '数据条数': 0,
                '开始日期': None,
                '结束日期': None,
                '状态': '空数据'
            })
            print("✗ 空数据")
            
        time.sleep(0.5)  # 避免请求过快
        
    except Exception as e:
        results.append({
            '品种代码': symbol,
            '品种名称': name,
            '数据条数': 0,
            '开始日期': None,
            '结束日期': None,
            '状态': str(e)[:50]
        })
        print(f"✗ {str(e)[:30]}")

# 显示结果汇总
results_df = pd.DataFrame(results)
print("\n获取结果汇总:")
display(results_df)

## 3. 测试有色金属品种

In [ ]:
# 测试有色金属
nonferrous_symbols = {
    'CU0': '铜',
    'AL0': '铝',
    'ZN0': '锌',
    'PB0': '铅',
    'NI0': '镍',
    'SN0': '锡'
}

nonferrous_results = []

for symbol, name in nonferrous_symbols.items():
    try:
        print(f"获取 {symbol} ({name})...", end=' ')
        df = ak.futures_zh_daily_sina(symbol=symbol)
        
        if df is not None and not df.empty:
            # 计算基本统计
            latest_price = df['close'].iloc[-1]
            price_range = f"{df['close'].min():.0f}-{df['close'].max():.0f}"
            
            nonferrous_results.append({
                '品种': f"{symbol} ({name})",
                '数据条数': len(df),
                '最新收盘': latest_price,
                '价格范围': price_range,
                '状态': '✓'
            })
            print(f"✓ 最新价: {latest_price:.0f}")
        else:
            nonferrous_results.append({
                '品种': f"{symbol} ({name})",
                '数据条数': 0,
                '最新收盘': None,
                '价格范围': None,
                '状态': '✗'
            })
            print("✗ 无数据")
            
        time.sleep(0.5)
        
    except Exception as e:
        nonferrous_results.append({
            '品种': f"{symbol} ({name})",
            '数据条数': 0,
            '最新收盘': None,
            '价格范围': None,
            '状态': '✗'
        })
        print(f"✗ 错误")

# 显示有色金属结果
nonferrous_df = pd.DataFrame(nonferrous_results)
print("\n有色金属获取结果:")
display(nonferrous_df)

## 4. 测试贵金属品种

In [ ]:
# 测试贵金属
precious_symbols = {
    'AU0': '黄金',
    'AG0': '白银'
}

print("贵金属品种测试:")
print("-" * 40)

for symbol, name in precious_symbols.items():
    try:
        df = ak.futures_zh_daily_sina(symbol=symbol)
        
        if df is not None and not df.empty:
            print(f"\n{symbol} ({name}):")
            print(f"  数据条数: {len(df)}")
            print(f"  日期范围: {df['date'].min()} 至 {df['date'].max()}")
            print(f"  最新收盘: {df['close'].iloc[-1]:.2f}")
            
            # 显示最近价格走势
            recent = df.tail(5)[['date', 'close', 'volume']]
            print("\n  最近5日走势:")
            display(recent)
        else:
            print(f"\n{symbol} ({name}): 无数据")
            
        time.sleep(0.5)
        
    except Exception as e:
        print(f"\n{symbol} ({name}): 获取失败 - {str(e)[:50]}")

## 5. 数据质量检查

In [ ]:
# 详细检查一个品种的数据质量
test_symbol = 'RB0'
print(f"数据质量检查: {test_symbol}")
print("=" * 50)

df = ak.futures_zh_daily_sina(symbol=test_symbol)

# 1. 列信息
print("\n1. 数据列信息:")
print(f"   列名: {list(df.columns)}")
print(f"   数据类型:")
for col in df.columns:
    print(f"     {col}: {df[col].dtype}")

# 2. 缺失值检查
print("\n2. 缺失值检查:")
missing = df.isnull().sum()
for col, count in missing.items():
    if count > 0:
        print(f"   {col}: {count} 个缺失值")
if missing.sum() == 0:
    print("   ✓ 无缺失值")

# 3. 数据范围检查
print("\n3. 价格数据范围:")
price_cols = ['open', 'high', 'low', 'close']
for col in price_cols:
    print(f"   {col}: {df[col].min():.2f} - {df[col].max():.2f}")

# 4. 数据一致性检查
print("\n4. 数据一致性检查:")
# 检查 high >= low
invalid_hl = (df['high'] < df['low']).sum()
print(f"   High < Low 的记录数: {invalid_hl}")

# 检查 high >= close 和 low <= close
invalid_hc = (df['high'] < df['close']).sum()
invalid_lc = (df['low'] > df['close']).sum()
print(f"   High < Close 的记录数: {invalid_hc}")
print(f"   Low > Close 的记录数: {invalid_lc}")

if invalid_hl + invalid_hc + invalid_lc == 0:
    print("   ✓ 数据一致性检查通过")

# 5. 统计信息
print("\n5. 基本统计信息:")
display(df[price_cols].describe())

## 6. 测试总结

In [ ]:
# 测试所有14个品种
all_symbols = [
    'AG0', 'AL0', 'AU0', 'CU0', 'HC0', 'I0', 'NI0', 
    'PB0', 'RB0', 'SF0', 'SM0', 'SN0', 'SS0', 'ZN0'
]

print("测试所有14个金属期货品种")
print("=" * 50)

success_count = 0
failed_list = []

for i, symbol in enumerate(all_symbols, 1):
    try:
        print(f"[{i:2}/14] {symbol}...", end=' ')
        df = ak.futures_zh_daily_sina(symbol=symbol)
        
        if df is not None and len(df) > 0:
            print(f"✓ {len(df):4} 条数据")
            success_count += 1
        else:
            print("✗ 空数据")
            failed_list.append(symbol)
            
        time.sleep(0.3)
        
    except Exception as e:
        print(f"✗ 错误")
        failed_list.append(symbol)

# 测试结果汇总
print("\n" + "=" * 50)
print(f"测试结果: {success_count}/14 个品种成功")

if failed_list:
    print(f"\n失败的品种: {failed_list}")
else:
    print("\n✓ 所有品种数据获取成功！")
    print("\n可以开始运行 01_data_fetch_fixed.ipynb 进行完整的数据管理")